**RNN - Recurrent Neural Network**

An RNN (Recurrent Neural Network) is a type of neural network that works great for sequential data — stuff like text, time-series, or anything where order matters.

In a regular neural net, we look at inputs all at once. But an RNN looks at things one step at a time, remembering things along the way using its hidden state — like a little memory.

In [ ]:
import torch
import torch.nn as nn
from torch.nn.functional import one_hot

**Character Preprocessing**

In [ ]:
# Character mapping
chars = sorted(list(set("hello")))
char2idx = {ch: i for i, ch in enumerate(chars)}
idx2char = {i: ch for ch, i in char2idx.items()}
vocab_size = len(chars)

# Convert sequence to indices
seq = "hello"
input_seq = [char2idx[ch] for ch in seq[:-1]]
target_seq = [char2idx[ch] for ch in seq[1:]]

# Convert to tensors
input_tensor = one_hot(torch.tensor(input_seq), num_classes=vocab_size).float().unsqueeze(0)
target_tensor = torch.tensor(target_seq)

**RNN Model**


In [ ]:
class SimpleRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(SimpleRNN, self).__init__()
    self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x, h0):
    out, hn = self.rnn(x, h0)
    out = self.fc(out)
    return out, hn

hidden_size = 16
model = SimpleRNN(vocab_size, hidden_size, vocab_size)

**Train the Model**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 200

for epoch in range(num_epochs):
  h0 = torch.zeros(1, 1, hidden_size)
  output, hn = model(input_tensor, h0)

  loss = criterion(output.view(-1, vocab_size), target_tensor)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if (epoch+1) % 20 == 0:
    pred = output.argmax(dim=2).squeeze().tolist()
    pred_chars = [idx2char[i] for i in pred]
    print(f"Epoch {epoch+1}: Loss= {loss.item():.4f}, Prediction: {''.join(pred_chars)}")

Epoch 20: Loss= 0.3261, Prediction: ello
Epoch 40: Loss= 0.0292, Prediction: ello
Epoch 60: Loss= 0.0095, Prediction: ello
Epoch 80: Loss= 0.0058, Prediction: ello
Epoch 100: Loss= 0.0043, Prediction: ello
Epoch 120: Loss= 0.0033, Prediction: ello
Epoch 140: Loss= 0.0027, Prediction: ello
Epoch 160: Loss= 0.0022, Prediction: ello
Epoch 180: Loss= 0.0019, Prediction: ello
Epoch 200: Loss= 0.0016, Prediction: ello


**Build the RNN-based Name Predictor**

In [ ]:
import torch
import torch.nn as nn
from torch.nn.functional import one_hot
from torch.utils.data import Dataset, DataLoader

In [ ]:
# The baby name dataset
names = ["alice", "alvin", "alex", "albert", "alena"]
names = [name + "#" for name in names]

# Create char dictionary
chars = sorted(list(set("".join(names))))
character2idx = {ch: i for i, ch in enumerate(chars)}
idx2character = {i: ch for ch, i in character2idx.items()}
vocab_size = len(chars)

**Custom Dataset for Character Sequences**

In [ ]:
class NameDataset(Dataset):
  def __init__(self, names, seq_len = 4):
    self.data = []
    for name in names:
      for i in range(1, len(name)):
        input_seq = name[:i]
        target_seq = name[1:i+1]
        self.data.append((input_seq, target_seq))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    inp, tgt = self.data[idx]
    inp_ids = [character2idx[ch] for ch in inp]
    tgt_ids = [character2idx[ch] for ch in tgt]

    input_tensor = one_hot(torch.tensor(inp_ids), num_classes=vocab_size).float()
    target_tensor = torch.tensor(tgt_ids)
    return input_tensor, target_tensor

dataset = NameDataset(names)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

**RNN Model**

In [ ]:
class NameRNN(nn.Module):
  def __init__(self, vocab_size, hidden_size):
    super(NameRNN, self).__init__()
    self.rnn = nn.RNN(vocab_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, vocab_size)

  def forward(self, x, h0=None):
    out, h = self.rnn(x, h0)
    out = self.fc(out)
    return out, h

model = NameRNN(vocab_size=vocab_size, hidden_size=32)

**Training the Model**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(300):
  total_loss = 0
  for inputs, targets in loader:
    h0 =  torch.zeros(1, 1, 32)
    output, _ = model(inputs, h0)

    loss = criterion(output.view(-1, vocab_size), targets.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

  if (epoch + 1) % 50 == 0:
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")


Epoch 50 Loss: 11.2250
Epoch 100 Loss: 11.9426
Epoch 150 Loss: 10.9508
Epoch 200 Loss: 10.2223
Epoch 250 Loss: 10.4068
Epoch 300 Loss: 10.4851


Prediction Function — **Autocomplete**!

In [ ]:
def predict(start, max_len=10):
    model.eval()
    with torch.no_grad():
        chars_input = [character2idx[ch] for ch in start]
        input_tensor = one_hot(torch.tensor(chars_input), num_classes=vocab_size).float().unsqueeze(0)
        h = None
        result = start

        for _ in range(max_len):
            output, h = model(input_tensor, h)
            pred_id = output[0, -1].argmax().item()
            pred_char = idx2character[pred_id]

            if pred_char == "#":
                break

            result += pred_char

            next_input = one_hot(torch.tensor([[pred_id]]), num_classes=vocab_size).float()
            input_tensor = next_input

        return result


In [ ]:
print(predict("al"))
print(predict("alb"))
print(predict("ale"))
print(predict("ali"))

alena
albert
alena
alice
